# Tratamento de dados

## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### objetivo
Deixar uma base organizada para poder estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.



## Vamos analisar as colunas 'ESCMAE', 'GESTACAO' e 'GRAVIDEZ', para revisar seus dados faltantes.

No caso da escolaridade da mãe, os valores faltantes podem ser considerados como uma nova categoria de 'sem informação', então não serão tratados de outra forma.

No caso do atributo de tempo de gestação e no tipo de gravidez, os valores faltantes realmente são um buraco nos dados pois têm muito valor para a análise, assim, tais linhas serão excluidas.

- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.

In [28]:
%pip install pandas
%pip install pandasql
%pip install --upgrade 'sqlalchemy<2.0'
import pandas as pd
import numpy as np
from pandasql import sqldf
df = pd.read_csv('SINASC_RO_2019.csv')
df.columns

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=705b3a300fcf640fab1a1ee7965e8036e17f41ef650343012d48be8dc729f380
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.29
    Uninstalling SQLAlchemy-2.0.29:
      Successfully uninstalled SQLAlchemy-2.0.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.


Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

## Contando o número de registros e o número de registros não duplicados da base.

In [4]:
total_registros = len(df)
df_sem_duplicados = df.drop_duplicates()
registros_nao_duplicados = len(df_sem_duplicados)
ha_duplicados = total_registros != registros_nao_duplicados
print("Número total de registros:", total_registros)
print("Número de registros não duplicados:", registros_nao_duplicados)
print("Há linhas duplicadas na base de dados:", ha_duplicados)


Número total de registros: 27028
Número de registros não duplicados: 27028
Há linhas duplicadas na base de dados: False


## Contando o número de valores missing por variável.

In [5]:
valores_ausentes_por_variavel = df.isna().sum()
print("Valores ausentes por variável:")
print(valores_ausentes_por_variavel)

Valores ausentes por variável:
ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


## Criando uma seleção dessa base somente com as colunas que interessam.

In [7]:
colunas_de_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
df_selecionado = df.loc[:, colunas_de_interesse]
print(df_selecionado.head())

   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO         GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0  37 a 41 semanas   
1        1        29        2.0      8 a 11 anos         1.0  37 a 41 semanas   
2        1        37        9.0      8 a 11 anos         2.0  37 a 41 semanas   
3        1        30        5.0  12 anos ou mais         0.0  37 a 41 semanas   
4        1        30        2.0      8 a 11 anos         1.0  37 a 41 semanas   

  GRAVIDEZ  CONSULTAS  APGAR5  
0    Única          4    10.0  
1    Única          4     9.0  
2    Única          4    10.0  
3    Única          3    10.0  
4    Única          4    10.0  


## Contando novamente o número de linhas e o número de missings

In [20]:
df_sem_missing_apgar5 = df_selecionado.dropna(subset=['APGAR5'])
num_registros_apgar5 = len(df_sem_missing_apgar5)
num_missing_apgar5 = df_sem_missing_apgar5['APGAR5'].isna().sum()
print("Número de linhas após a remoção de registros com APGAR5 não preenchido:", num_registros_apgar5)
print("Número de valores ausentes na coluna 'APGAR5' após a remoção dos registros:", num_missing_apgar5)

Número de linhas após a remoção de registros com APGAR5 não preenchido: 26925
Número de valores ausentes na coluna 'APGAR5' após a remoção dos registros: 0


## Nas colunas 'ESTCIVMAE' e 'CONSULTAS', irei substituir o 9 por Not a Number do Pandas

In [25]:
df['ESTCIVMAE'].replace(9, pd.NA)
df['CONSULTAS'].replace(9, pd.NA)


0        4
1        4
2        4
3        3
4        4
        ..
27023    4
27024    4
27025    4
27026    4
27027    4
Name: CONSULTAS, Length: 27028, dtype: object

## Substituindo os valores faltantes da quantitativa (QTDFILVIVO) por zero.


In [27]:
df['QTDFILVIVO'].fillna(0)
print(df['QTDFILVIVO'].isna().sum())
print(df.head())

0
   ORIGEM   CODESTAB  CODMUNNASC  LOCNASC  IDADEMAE ESTCIVMAE  \
0       1  2679477.0      110001        1        19       5.0   
1       1  2679477.0      110001        1        29       2.0   
2       1  2679477.0      110001        1        37      <NA>   
3       1  2516500.0      110001        1        30       5.0   
4       1  2516500.0      110001        1        30       2.0   

            ESCMAE  CODOCUPMAE  QTDFILVIVO  QTDFILMORT  ...  KOTELCHUCK  \
0      8 a 11 anos         NaN         0.0         0.0  ...           5   
1      8 a 11 anos    999992.0         1.0         0.0  ...           5   
2      8 a 11 anos    513205.0         2.0         0.0  ...           5   
3  12 anos ou mais    231205.0         0.0         0.0  ...           4   
4      8 a 11 anos    999992.0         1.0         0.0  ...           5   

  CONTADOR munResStatus munResTipo               munResNome  munResUf  \
0        1        ATIVO      MUNIC    Alta Floresta D'Oeste  Rondônia   
1        2

## Substituindo valores NaN ou faltantes pela string 'ausente'


Vou substituir os valores faltantes por "ausente" (string), a fim de manter certa integridade dos dados. Optamos por substituir po rum alabel apenas para representar o dado no formato object (string), e para nao termos problemas com calculos, ja que NaN pode assumir o valor de float.


In [33]:
df.fillna('ausente', inplace=True)
print(df.isna().sum())

ORIGEM        0
CODESTAB      0
CODMUNNASC    0
LOCNASC       0
IDADEMAE      0
             ..
munResUf      0
munResLat     0
munResLon     0
munResAlt     0
munResArea    0
Length: 69, dtype: int64


## Criando uma categorização da variável 'apgar' com essa codificação e calcule as frequências dessa categorização

In [29]:
notas = sqldf('''SELECT
    COUNT(CASE WHEN apgar1 BETWEEN 8 AND 10 THEN 1 END) AS normal_apgar1,
    COUNT(CASE WHEN apgar1 BETWEEN 6 AND 7 THEN 1 END) AS leve_apgar1,
    COUNT(CASE WHEN apgar1 BETWEEN 4 AND 5 THEN 1 END) AS moderado_apgar1,
    COUNT(CASE WHEN apgar1 BETWEEN 0 AND 3 THEN 1 END) AS severo_apgar1,
    COUNT(CASE WHEN apgar5 BETWEEN 8 AND 10 THEN 1 END) AS normal_apgar5,
    COUNT(CASE WHEN apgar5 BETWEEN 6 AND 7 THEN 1 END) AS leve_apgar5,
    COUNT(CASE WHEN apgar5 BETWEEN 4 AND 5 THEN 1 END) AS moderado_apgar5,
    COUNT(CASE WHEN apgar5 BETWEEN 0 AND 3 THEN 1 END) AS severo_apgar5
FROM
df
''')
notas

,normal_apgar1,leve_apgar1,moderado_apgar1,severo_apgar1,normal_apgar5,leve_apgar5,moderado_apgar5,severo_apgar5
0,23802,2523,376,231,26463,320,68,74


## Renomeando as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras

In [35]:
df_selecionado.rename(columns={
    'LOCNASC': 'local_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5'
}, inplace =  True)